## 1. Set up Environment

In [1]:
import pandas as pd
import requests
import datetime
import pyodbc
from sqlalchemy import create_engine

## 2. Store API Key

In [2]:
key = open('alphavantage_key.txt').read()

## 3. Construct General Form for API Call

In [3]:
class api_construct:
    def __init__(self, function, symbol, apikey):
        self.function = function
        self. symbol = symbol
        self.apikey = apikey

    def intraday(self, interval='1min', adjusted='true', outputsize='compact', datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&interval=' + interval + '&adjusted=' + adjusted + '&outputsize=' + outputsize + '&apikey='\
        + self.apikey + '&datatype=' + datatype
        
    def intraday_ext(self, interval='1min', slice_='year1month1', adjusted='true'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&interval=' + interval + '&slice=' + slice_ + '&adjusted=' + adjusted + '&apikey=' + self.apikey
        
    def daily_adj(self, outputsize='compact', datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&outputsize=' + outputsize + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def weekly(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def weekly_adj(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
    
    def monthly(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def monthly_adj(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype

def api_call(function, symbol, **kwargs):
    construct = api_construct(function, symbol, key)
    
    if function == 'TIME_SERIES_INTRADAY':
        construct.intraday(**kwargs)
        tbl_name = 'intraday'
    elif function == 'TIME_SERIES_INTRADAY_EXTENDED':
        construct.intraday_ext(**kwargs)
        tbl_name = 'intraday_ext'
    elif function == 'TIME_SERIES_DAILY_ADJUSTED':
        construct.daily_adj(**kwargs)
        tbl_name = 'daily_adj'
    elif function == 'TIME_SERIES_WEEKLY':
        construct.weekly(**kwargs)
        tbl_name = 'weekly'
    elif function == 'TIME_SERIES_WEEKLY_ADJUSTED':
        construct.weekly_adj(**kwargs)
        tbl_name = 'weekly_adj'
    elif function == 'TIME_SERIES_MONTHLY':
        construct.monthly(**kwargs)
        tbl_name = 'monthly'
    elif function == 'TIME_SERIES_MONTHLY_ADJUSTED':
        construct.monthly_adj(**kwargs)
        tbl_name = 'monthly_adj'
    else:
        print('Invalid function input')
        
    url = construct.url
    return url, tbl_name

## 4. Make the API Call 

In [4]:
x = api_call('TIME_SERIES_DAILY_ADJUSTED', 'TRKA', outputsize='compact')
url = x[0]
tbl_name = x[1]
r = requests.get(url)
print(f"Status code: {r.status_code}")  # Status Code 200 means the API call was successful
data = r.json()

# How to deal with csv file?

Status code: 200


In [5]:
data

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events',
  '2. Symbol': 'TRKA',
  '3. Last Refreshed': '2023-05-10',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2023-05-10': {'1. open': '0.1901',
   '2. high': '0.1901',
   '3. low': '0.1849',
   '4. close': '0.187',
   '5. adjusted close': '0.187',
   '6. volume': '7309792',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-05-09': {'1. open': '0.1892',
   '2. high': '0.1931',
   '3. low': '0.1847',
   '4. close': '0.1901',
   '5. adjusted close': '0.1901',
   '6. volume': '7437083',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-05-08': {'1. open': '0.195',
   '2. high': '0.1954',
   '3. low': '0.1871',
   '4. close': '0.1906',
   '5. adjusted close': '0.1906',
   '6. volume': '9489638',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'},
  '2023-05-05': {'1. open': '0.1859',
   '2. hi

## 5. Format the data and put it into SQL Server

In [6]:
# json outputs a multi-dimensional dictionary where the second outer key is our time series name. Use list() to extract it
tseries_keyname = list(data.keys())[1]

# Place the json output into dictionary format. 
dicts, meta = data[tseries_keyname], data['Meta Data']

try:
    del interval  # In case of code run without for a function w/o interval after previous run included interval
except Exception:
    pass

try:
    symbol = meta['2. Symbol']
    interval = meta['4. Interval']
except KeyError:
    symbol = meta['2. Symbol']

# Create empty list to store what will go into our SQL Table. We are bulk inserting using cursor.executemany()
# which only takes two arguments - (insert statement, entire list of values)
values = []

In [7]:
tbl_keys = list( dicts[ list(dicts.keys())[0] ].keys() )

# dicts is a multi-dimensional dictionary where 'date' is each key with multiple values:
try:
    i = 0
    for date in dicts:
        values.append((f"{symbol}_{date}",symbol,date, interval))
        for key in tbl_keys:
            values[i] = values[i] + tuple( [float(dicts[date][key])] )  
            # Note that we can concatenate tuples and use tuple() to change list to tuple
        i += 1
except NameError:
    i = 0
    for date in dicts:
        values.append((f"{symbol}_{date}",symbol,date))
        for key in tbl_keys:
            values[i] = values[i] + tuple( [float(dicts[date][key])] )  
            # Note that we can concatenate tuples and use tuple() to change list to tuple
        i += 1
        
        
'''
# OLD way that is NOT in General Form
for date in dicts:      
    values.append((f"{symbol}_{date}",symbol,date,float(dicts[date]['1. open']),float(dicts[date]['2. high']),\
                  float(dicts[date]['3. low']),float(dicts[date]['4. close']),float(dicts[date]['5. adjusted close']),\
                  int(dicts[date]['6. volume']),float(dicts[date]['7. dividend amount']),\
                  float(dicts[date]['8. split coefficient'])\
                  )\
                 )
''' 

'\n# OLD way that is NOT in General Form\nfor date in dicts:      \n    values.append((f"{symbol}_{date}",symbol,date,float(dicts[date][\'1. open\']),float(dicts[date][\'2. high\']),                  float(dicts[date][\'3. low\']),float(dicts[date][\'4. close\']),float(dicts[date][\'5. adjusted close\']),                  int(dicts[date][\'6. volume\']),float(dicts[date][\'7. dividend amount\']),                  float(dicts[date][\'8. split coefficient\'])                  )                 )\n'

### 5.1 Make the ODBC Connection

In [8]:
# ODBC Connection only needs to happen once, but it needs to happen during every new session
SERVER = 'DESKTOP-FKKJE0E'
DATABASE = 'StockData'
DRIVER = 'ODBC Driver 17 for SQL Server'
USERNAME = 'sa'
PASSWORD = '39xkjrj#OY'

conn = pyodbc.connect(f'DRIVER={DRIVER};\
                        SERVER={SERVER};\
                        DATABASE={DATABASE};\
                        UID={USERNAME};PWD={PASSWORD}')
cursor = conn.cursor()                

### 5.2 Insert the data into SQL Table

In [9]:
# To deal with duplicate values, we check for existence by creating a ## global temporary table to store the current runtime
# data and compare it with the existing table in SQL. Insert values that don't already exist

cursor.execute("DROP TABLE IF EXISTS StockData.dbo.##tempstock_tbl;")
try:
    interval
    cursor.execute("CREATE TABLE StockData.dbo.##tempstock_tbl\
        (stock_id VARCHAR(255), symbol VARCHAR(15), [date] DATETIME, interval VARCHAR(10));")
except NameError:
    cursor.execute("CREATE TABLE StockData.dbo.##tempstock_tbl\
        (stock_id VARCHAR(255), symbol VARCHAR(15), [date] DATE);")

# Dynamically add the number and names of column headers depending on which time series is used
for i in tbl_keys:
    colName = "[" + i + "]"    # We MUST surround our column title with [] just in case it is a RESERVED word
    cursor.execute("EXEC StockData.dbo.addcol_tempstock @colName = ?",(colName))
    
headerStr = ''
for i in tbl_keys:
    colName = "[" + i + "]"
    headerStr = headerStr + colName + ','
headerStr = headerStr[:-1]

cursor.execute("SELECT COUNT(COLUMN_NAME) FROM StockData.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '" + tbl_name + "';")
colSize = str(cursor.fetchone())
colSize = colSize.replace("(","")
colSize = colSize.replace(")","")
colSize = colSize.replace(",","")
colSize = int(colSize)
xValues = ""
for i in range(0, colSize):
    xValues = xValues + "?,"
xValues = xValues[:-1]    


# OUR DATE VALUES can't have the extra hours, minutes, seconds because our table in SQL server doesn't use these switch to 
# datetime???
try:
    interval
    cursor.fast_executemany = True
    cursor.executemany("INSERT INTO StockData.dbo.##tempstock_tbl (stock_id, symbol, [date], interval, " + headerStr + ")\
                  VALUES (" + xValues + ")",\
                  values)
except NameError:
    cursor.fast_executemany = True
    cursor.executemany("INSERT INTO StockData.dbo.##tempstock_tbl (stock_id, symbol, [date], " + headerStr + ")\
                  VALUES (" + xValues + ")",\
                  values)
    
cursor.execute("INSERT INTO StockData.dbo." + tbl_name + " SELECT * FROM StockData.dbo.##tempstock_tbl\
                WHERE stock_id NOT IN (SELECT stock_id FROM StockData.dbo." + tbl_name + ")")
conn.commit()